## Les librairies

In [155]:
import pandas as pd
import nltk
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
import requests
import string
import pdfplumber
from pdfminer.psparser import PSEOF
from pdfminer.pdfdocument import PDFSyntaxError
import os
import shutil
import time
import struct
import glob

In [156]:
import time
from multiprocessing import Pool, cpu_count, Process, Queue
import threading
from tqdm import tqdm
from colorama import Fore

In [157]:
import fitz
from PIL import Image

In [158]:
from decimal import Decimal, DecimalException, InvalidOperation

In [159]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

## Transfert vers drive

In [160]:
#Récupérer le json confidentiel à https://developers.google.com/drive/activity/v2/quickstart/python
#Renommer le fichier client_secrets.json

gauth = GoogleAuth()
gauth.LocalWebserverAuth()

drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=523991296033-q2rufardngpng9dprm10li4j2tsg3uh0.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [161]:
def obtenir_id(nom_file):
    # Auto-iterate through all files that matches this query
    file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
    for file1 in file_list:
        if file1['title'] == nom_file:
            return file1['id']

In [162]:
def mis_en_line(nom):
    
    #Architecture en ligne
    folder = drive.CreateFile({'title' : nom, 'parents': [{'id': obtenir_id("Thèse extraite")}], 'mimeType' : 'application/vnd.google-apps.folder'})
    folder.Upload()
    identite = folder['id']
    
    filenames = os.listdir("Thèse extraite/" + nom + "/")
    liste_texte_ok = []
    for fname in filenames:
        if (fname != "Page phrase colle") and (fname != "Image"):
            with open("Thèse extraite/" + nom + "/" + fname) as infile:
                num_page = fname.replace("Page ", "")
                num_page = num_page.replace(".txt", "")
                liste_texte_ok.insert(0, (infile.read(), int(num_page)))
    
    liste_texte_ok = sorted(liste_texte_ok, key=lambda texte: texte[1])
    liste_ok_final = []
    for i in liste_texte_ok:
        liste_ok_final.append(i[0])

    filenames = os.listdir("Thèse extraite/" + nom + "/Page phrase colle/")
    liste_texte_colle = []
    for fname in filenames:
        with open("Thèse extraite/" + nom + "/Page phrase colle/" + fname) as infile:
            liste_texte_colle.insert(0, infile.read())
    
    texte_ok = "\n \n _LLAMMA_BIEN_LU \n \n".join(liste_ok_final)
    texte_colle = "\n \n _LLAMMA_COLLE \n \n".join(liste_texte_colle)
    
    #Ecriture sur drive
    these_bien_lu = drive.CreateFile({'title' : "Thèse " + nom + "ok", 'parents': [{'id': identite}] ,'mimeType' : 'text/plain'})
    these_bien_lu.SetContentString(texte_ok)
    
    these_lu_colle = drive.CreateFile({'title' : "Thèse " + nom + "colle", 'parents': [{'id': identite}] ,'mimeType' : 'text/plain'})
    these_lu_colle.SetContentString(texte_colle)
    
    these_bien_lu.Upload()
    these_lu_colle.Upload()

## Pré-traitement 


In [163]:
def creation_arborescence_mobile(nom, traitement_image):
    global dossier_destination

    #Dossier pour les pages bien lues
    if(not os.path.exists(dossier_destination + "/" + nom)):
        os.makedirs(dossier_destination + "/" + nom)

    #Dossier pour mettre les pages avec les phrases collées 
    if(not os.path.exists(dossier_destination + "/" + nom + "/Page phrase colle")):
        os.makedirs(dossier_destination + "/" + nom + "/Page phrase colle")
    
    if(traitement_image):
        #Répertoire pour mettre les images
        if(not os.path.exists(dossier_destination + "/" + nom + "/Image")):
            os.makedirs(dossier_destination + "/" + nom + "/Image")

In [164]:
def creation_arborescence_fixe():
    global dossier_destination, dossier_mal_lu
    
    if(not os.path.exists(dossier_destination)):
        os.makedirs(dossier_destination)
    
    if(not os.path.exists(dossier_destination + "/Français")):
        os.makedirs(dossier_destination + "/Français")
        
    if(not os.path.exists(dossier_destination + "/Anglais")):
        os.makedirs(dossier_destination + "/Anglais")
        
    if(not os.path.exists(dossier_mal_lu)):
        os.makedirs(dossier_mal_lu)
        
    if(not os.path.exists(dossier_mal_lu + "/Probleme page vide")):
        os.makedirs(dossier_mal_lu + "/Probleme page vide")
        
    if(not os.path.exists(dossier_mal_lu + "/Probleme thèse mal encodée")):
        os.makedirs(dossier_mal_lu + "/Probleme thèse mal encodée")
        
    if(not os.path.exists(dossier_mal_lu + "/Trop peu lu")):
        os.makedirs(dossier_mal_lu + "/Trop peu lu")
        
    if(not os.path.exists(dossier_mal_lu + "/Problème phrase collées")):
        os.makedirs(dossier_mal_lu + "/Problème phrase collées")
    
    #Architecture fixe en ligne
    if obtenir_id("Thèse extraite") == None:
        folder = drive.CreateFile({'title' : "Thèse extraite", 'mimeType' : 'application/vnd.google-apps.folder'})
        folder.Upload()

## Nettoyage primaire

In [165]:
#Permet de vérifier si une page à 5 caractères différents
def controle_page_valide(page):
    
    chiffre = "0123456789"
    liste_chiffre = list(chiffre)
    total_caractere = list(string.ascii_lowercase)

    A_controler = list(page)
    
    compteur = 0

    for lettre_page in A_controler:
        for lettre_controle in range(len(total_caractere) - 1):
            if(total_caractere[lettre_controle] == lettre_page):
                compteur = compteur + 1
                del total_caractere[lettre_controle]
                
    if(compteur >= 5):
        #Indique le nombre de caractères différents dans la page
        return False
    else:
        return True

In [166]:
def traitement_texte(texte):
    texte = texte.replace("`", "")
    texte = texte.replace("ˆ", "")
    texte = texte.replace("´", "")

    for a in range(500): 
        texte = texte.replace("(cid:{num})".format(num = a), "")
        
    return texte

## Traitement phrase colle

In [1]:
def compare_mot(mot_teste, phrase_decoupe):
    
    global liste_mot
    
    if(mot_teste == phrase_decoupe):
        return True
    
    phrase_decoupe_inter = phrase_decoupe[len(mot_teste):]
    
    for mot in liste_mot:
        mot_decomp = list(mot)
        
        compteur = 0
        while ((compteur < len(phrase_decoupe_inter)) and (compteur < len(mot_decomp))):
            
            if(mot_decomp[compteur] == phrase_decoupe_inter[compteur]):
                compteur = compteur + 1

                if(compteur == len(mot_decomp)):
                    return True
            else :
                break

    return False

In [2]:
def decoupe_phrase_mot_long(phrase):
    global liste_mot
    
    phrase_decoupe = list(phrase)
    phrase_decoupe_finale = []

    while len(phrase_decoupe)>0:
        
        mot_retenu = []
        longueur_mot_retenu = 0
        
        #On parcourt l'ensemble des mots du dictionnaire
        for mot in liste_mot:
            mot_decomp = list(mot)
            
            compteur = 0
            while ((compteur < len(phrase_decoupe)) and (compteur < len(mot_decomp))):
                
                if mot_decomp[compteur] == " ":
                    return " ".join(phrase_decoupe_finale)
                
                #Traitement d'un mot dans une phrase
                if((mot_decomp[compteur] == phrase_decoupe[compteur])):
                    compteur = compteur + 1
                    
                    #DANGER
                    if(compteur == len(mot_decomp)) and (longueur_mot_retenu < len(mot_decomp)) and compare_mot("".join(mot_decomp), "".join(phrase_decoupe)):
                        mot_retenu = mot_decomp
                        longueur_mot_retenu = len(mot_retenu)
                else :
                    break

        #Mise à jour de la phrase dans le cas où aucun mot n'a été trouvé 
        if(len(mot_retenu) == 0):
            del phrase_decoupe[0]
        
        #Mise à jour de la phrase dans le cas où un mot a été trouvé
        else :
            for i in range(len(mot_retenu)):
                del phrase_decoupe[0]

            mot_retenu = "".join(mot_retenu)
            phrase_decoupe_finale.append(mot_retenu)
    
    
    return " ".join(phrase_decoupe_finale)

In [169]:
def traitement_phrase_colle(texte, nombre_de_lettre_colle = 25, phrase_colle_page = 3):
    
    page_splite = texte.replace(".", " ")
    page_splite = page_splite.replace("_", " ")
    page_splite = page_splite.split(" ")
    compteur_colle = 0
    
    for mot in range(len(page_splite)) :
        if len(page_splite[mot])>nombre_de_lettre_colle :
            
            texte = texte.lower()
            texte = texte.replace(".", " .")
            texte = texte.replace("!", " !")
            texte = texte.replace("?", " ?")
            texte = texte.replace("\'", "")
            texte = texte.replace(",", " ")
            
            
            compteur_colle += 1
            #A PENSER DANS UN THREAD
            #page_splite[mot] = decoupe_phrase_mot_long(page_splite[mot])
    
    if compteur_colle > phrase_colle_page :
        return (True, compteur_colle)
    
    return (False, compteur_colle)

## Cas d'arrêt prématuré 

In [170]:
def timer_these(temps = 350):
    global arret_timer

    temps_passe = 0
    while temps_passe < temps and not arret_timer:
        time.sleep(1)
        temps_passe = temps_passe + 1
        
    if(temps_passe >= temps):
        raise TimeoutError

## Post traitement

In [171]:
def pourcentage_phrase_colle(nom):
    global dossier_destination
    
    nombre_page_bien_lues = len(os.listdir(dossier_destination + "/" + nom + "/")) - 1
    nombre_page_males_lues = len(os.listdir(dossier_destination + "/" + nom + "/Page phrase colle/"))
    
    return nombre_page_males_lues / (nombre_page_bien_lues + nombre_page_males_lues)

In [172]:
def condition_post_extraction(nom, mal_encode, compteur_page_vide):
    global seuil_nb_pages, seuil_page_vide, dossier_mal_lu, pourcentage_colle_refus, dossier_mal_lu, dossier_destination
    
    #Post traitement
    if os.path.exists(dossier_destination + "/" + nom):
        if(len(os.listdir(dossier_destination + "/" + nom)) < seuil_nb_pages):
            print("Trop peu de pages lues")
            shutil.rmtree(dossier_destination + "/" + nom)
            shutil.move(nom + ".pdf",dossier_mal_lu + "/Trop peu lu/" + nom + ".pdf")

        elif(compteur_page_vide > seuil_page_vide):
            print("Trop de page blanches")
            shutil.rmtree(dossier_destination + "/" + nom)
            shutil.move(nom + ".pdf",dossier_mal_lu + "/Probleme page vide/" + nom + ".pdf")

        elif(pourcentage_phrase_colle(nom) > pourcentage_colle_refus):
            print("Trop de page avec phrases collées")
            shutil.rmtree(dossier_destination + "/" + nom)
            shutil.move(nom + ".pdf",dossier_mal_lu + "/Problème phrase collées/" + nom + ".pdf")
        
    if mal_encode :
        print("Thèse non traduisible")
        fichier.close()
        shutil.rmtree(dossier_destination + "/" + nom)
        shutil.move(nom + ".pdf",dossier_mal_lu + "/Probleme thèse mal encodée/" + nom + ".pdf")

## Traitement texte

In [173]:
def extraction_remplissage(page, nom, queue, num_page):
    try:
        tuple_inserer = [page.extract_text(), nom, num_page]
    except (UnicodeError, UnicodeDecodeError, UnicodeEncodeError, TimeoutError, struct.error, TypeError, AssertionError):
        tuple_inserer = ["Erreur", nom, num_page]
    queue.put(tuple_inserer)

In [174]:
def assemblage(liste_paire, liste_impaire):
    liste_totale = []
    for texte_p, texte_imp in zip(liste_paire,liste_impaire):
        liste_totale.append(texte_p)
        liste_totale.append(texte_imp)
    return liste_totale

In [175]:
def traitement_pages(fichier_lu, traitement_image):
    global dossier_destination, seuil_lettre_phrase_colle, seuil_phrase_colle, temps_limite, seuil_nb_pages, seuil_page_vide, dossier_mal_lu, pourcentage_colle_refus, taille_max, nombre_de_page_illisible, compteur_page_vide, mal_encode, arret_timer
    
    print(fichier_lu)

    nom = fichier_lu.replace(".pdf", "")
    creation_arborescence_mobile(nom, traitement_image)

    #On lance le timer pour éviter les temps de traitement aberrant
    timer = threading.Thread(target=timer_these)
    timer.start()

    try:
        with pdfplumber.open(fichier_lu) as pdf :
            page_textes_paire = []
            page_textes_impaire = []

            queue_paire = Queue(len(pdf.pages))
            queue_impaire = Queue(len(pdf.pages))

            page = 0
            bar = tqdm(total = len(pdf.pages))
            while page < len(pdf.pages):
                p1 = Process(target=extraction_remplissage, args=(pdf.pages[page], nom, queue_paire, page,))
                p1.start()
                page_textes_paire.append(queue_paire.get())
                page += 1
                
                if (page < len(pdf.pages)) :
                    p2 = Process(target=extraction_remplissage, args=(pdf.pages[page], nom, queue_impaire, page,))
                    p2.start()
                    page_textes_impaire.append(queue_impaire.get())
                
                p1.join()
                if (page < len(pdf.pages)) :
                    p2.join()
                    page +=1
                    bar.update(1)

                bar.update(1)

                if not timer.is_alive() :
                    break
                
            bar.close()

            #On assemble
            page_texte = assemblage(page_textes_paire, page_textes_impaire)

            if (len(page_texte) == 0):
                print("Thèse vide ou scannée")

            #On parallélise l'écriture des pages
            with Pool() as pool:
                pool.map(extraction_page, page_texte)
                
        #Conditions avec post traitement
        condition_post_extraction(nom, mal_encode, compteur_page_vide)

        #Mise en ligne
        if os.path.exists(dossier_destination + "/" + nom):
            mis_en_line(nom)

    except (InvalidOperation,PSEOF ,TypeError, PDFSyntaxError, TimeoutError, BrokenPipeError) :
        print("Erreur de lecture")

    
    #Re-initilisation
    #Variable contrôle qualité page
    nombre_de_page_illisible = 0
    compteur_page_vide = 0
    numeros_de_page = 0
    mal_encode = False

In [176]:
def extraction_page(texte_nom):
    global nombre_de_page_illisible, compteur_page_vide, mal_encode

    texte = texte_nom[0]
    nom = texte_nom[1]
    numeros_de_page = texte_nom[2]
    
    if (not mal_encode):
        try :
            page_traite = traitement_texte(texte)

            #Traitement phrase collées
            traitement_colle = traitement_phrase_colle(page_traite, seuil_lettre_phrase_colle, seuil_phrase_colle)

            #On crée le fichier
            if not traitement_colle[0] :
                fichier = open(dossier_destination + "/" + nom +'/Page {page}.txt'.format(page=numeros_de_page), "a", encoding = "utf-8")

            else :
                fichier = open(dossier_destination + "/" + nom + '/Page phrase colle/Page {page} traite {nb_traite}.txt'.format(page=numeros_de_page, nb_traite = traitement_colle[1]), "a", encoding = "utf-8")

            #10 illisible sur les 30 premières
            if (len(os.listdir(dossier_destination + "/" + nom)) < 30):
                if (controle_page_valide(page_traite)):
                    nombre_de_page_illisible = nombre_de_page_illisible + 1

            if (nombre_de_page_illisible >= 10):
                mal_encode = True

            if not mal_encode:
                fichier.write(page_traite)


        except (AttributeError, FileNotFoundError):
            fichier = open(dossier_destination + "/" + nom + '/Page {page}.txt'.format(page=numeros_de_page), "a", encoding = "utf-8")
            fichier.write("Page Vide")
            compteur_page_vide = compteur_page_vide + 1

        #On clot le fichier et on supprime la page
        fichier.close()

## Traitement images

In [177]:
def controle_image(nom, largeur_min, hauteur_min):
    image=Image.open(nom)
    if image.size[0] < largeur_min or image.size[1] < hauteur_min:
        image.close()
        os.remove(nom)
    image.close()

In [178]:
def traitement_image(pdf_lu):
    
    global dossier_destination
    
    nom = pdf_lu.replace(".pdf", "")
    if os.path.exists(dossier_destination + "/" + nom):
        doc = fitz.open(pdf_lu)
        nom = pdf_lu.replace(".pdf", "")


        for i in range(len(doc)):
            for img in doc.getPageImageList(i):
                try :
                    xref = img[0]
                    pix = fitz.Pixmap(doc, xref)
                    chemin = dossier_destination + "/" + nom + "/Image/%s.png" % (i)
                    if pix.n < 5:
                        pix.writePNG(chemin)

                    else :
                        pix = fitz.Pixmap(fitz.csRGB, pix)
                        pix.writeImage(chemin)

                    controle_image(chemin, 350, 350)
                    pix = None

                except RuntimeError :
                    pix = None
                    os.remove(chemin)                
                    pass
                except FileNotFoundError:
                    pass

## Traitement de la langue

In [179]:
def Quelle_langue(document):
   
    mots_partages = {}
    mots = wordpunct_tokenize(document)
    for langue in stopwords.fileids():
        liste_stopWds = stopwords.words(langue)
        mots = set(mots)
       
        elements_communs = mots.intersection(liste_stopWds)
        mots_partages[langue] = len(elements_communs)
       
    return max(mots_partages, key= mots_partages.get)

In [180]:
def test_langue_sep(dossier_source):
    
    global compteur_français, compteur_anglais
    
    liste_theses = os.listdir(dossier_source)
    for dossier in liste_theses:
        if(dossier != "Français" and dossier != "Anglais"):
    
            Cpt_fr = 0
            Cpt_en = 0
            english = 0
            french = 0

            liste_fichiers = os.listdir(dossier_source +"/"+dossier)
            liste_theses = os.listdir(dossier_source)
            taille = len(liste_fichiers)

            for fichier in sorted(glob.glob(dossier_source + "/" + dossier + "/*.txt")):
                with open(fichier,'r') as txt:
                    Texte = txt.read()
                    if(Quelle_langue(Texte)=='french'):
                        Cpt_fr+=1
                    elif(Quelle_langue(Texte)=='english'):
                        Cpt_en+=1

            if((Cpt_fr/taille) >= .80):
                compteur_français += 1
                shutil.move(os.path.join(dossier_source,dossier),dossier_source+'/Français')

            elif((Cpt_en/taille) >= .80):
                compteur_anglais += 1
                shutil.move(os.path.join(dossier_source,dossier),dossier_source+'/Anglais')

            else:
                shutil.rmtree(os.path.join(dossier_source,dossier))   

## Code téléchargement

In [181]:
def telechargement(nom_src):

    url = 'https://tel.archives-ouvertes.fr/' + nom_src + '/document'
    try :
        r = requests.get(url, stream=True)
        with open(nom_src + '.pdf', 'wb') as fd:
            for chunk in r.iter_content(2000):
                fd.write(chunk)

    except ConnectionError:
        print("Erreur dans la connection")

## Paramètres

In [182]:
data = pd.read_csv("PhD_DB_Discip_recodée.csv", encoding="utf-8")
domaine_list = ['X', 'IV', 'V', 'VII', 'VI', 'VIII', 'III', 'IX', 'II', 'I - Droit', 'XII', 'I', 'pharmacie']
data = data.drop(["Defence.date","Domains", "n.disc", "disc1.lev2", "disc1.lev3", "disc2.lev1", "disc2.rec.lev1", "disc2.rec.lev2", "disc2.rec.lev3", "disc3.rec.lev1", "disc3.rec.lev2", "disc3.rec.lev3", "Author", "Author.ID", "views","downloads"], axis = 1)
data = data.drop(["disc1.lev1", "disc2.lev2", "disc2.lev3", "disc3.lev1", "disc3.lev2", "disc3.lev3", "n.tag", "disc1.rec.lev2", "disc1.rec.lev3"], axis = 1)
data = data.dropna(axis=0)

In [183]:
#Variable initiale de traitement texte
#Variable contrôle qualité page
nombre_de_page_illisible = 0
compteur_page_vide = 0
numeros_de_page = -1
mal_encode = False
arret_timer = False

#Nom du dossier destination
dossier_destination = "Thèse extraite"

#Nom du dossier destination pour les documents mals lus
dossier_mal_lu = "Thèse mal lues"

taille_max = 45000000
seuil_lettre_phrase_colle = 25
seuil_phrase_colle = 3
seuil_nb_pages = 10
seuil_page_vide = 40
pourcentage_colle_refus = 0.4


compteur_français = 0
compteur_anglais = 0

## Code principal

In [184]:
def traitement_total(nom_src, data, traitement_image):
    global dossier_destination
    
    if(os.path.exists(dossier_destination + "/" + nom_src)):
        shutil.rmtree(dossier_destination + "/" + nom_src)
    
    #Petit filtre par taille
    if (os.path.getsize(nom_src + '.pdf') < taille_max):
        traitement_pages(nom_src + ".pdf", traitement_image)
        
        if(traitement_image):
            traitement_image(nom_src + ".pdf")
    else :
        print("Thèse trop lourde")

In [190]:
def extraction(data, nb_process, traitement_image):
    global dossier_destination, taille_max, compteur_français, compteur_anglais
    
    creation_arborescence_fixe()
    temps = time.time()

    for indice_domaine in range(len(domaine_list)):
        
        #Différents compteurs et pour savoir l'avancée il faudra les sauvegarder qq part
        compteur_français = 0
        compteur_anglais = 0
        indice = 0
        
        
        if(not os.path.exists(domaine_list[indice_domaine] + " memoire.csv")):
            pd.DataFrame({domaine_list[indice_domaine] : [], "Etat" : []}).to_csv(domaine_list[indice_domaine] + " memoire.csv")

            
        #Mémoire pour revenir où on s'était arrêté
        memoire = list(pd.read_csv(domaine_list[indice_domaine] + " memoire.csv")[domaine_list[indice_domaine]])
        etat_traitement = list(pd.read_csv(domaine_list[indice_domaine] + " memoire.csv")["Etat"])
        
        
        data_domaine = data[data["def.date"] > 2000]
        data_domaine = data_domaine[data_domaine["disc1.rec.lev1"] == domaine_list[indice_domaine]]
        data_domaine.index = range(data_domaine.shape[0])

        while (compteur_français < 50) and (compteur_anglais < 50):

            #Création de Process
            liste_process = []
            for process in range(nb_process):
                
                
                while data_domaine["these.id"][indice] in list(pd.read_csv(domaine_list[indice_domaine] + " memoire.csv")[domaine_list[indice_domaine]]):
                    indice+=1
                    
                nom_src = data_domaine["these.id"][indice]
                telechargement(nom_src)

                liste_process.append((nom_src,Process(target = traitement_total, args=(nom_src, data, traitement_image, ))))
                liste_process[process][1].start()
                indice+=1

            for process_en_cours in liste_process :
                process_en_cours[1].join()
                memoire.append(process_en_cours[0])
                
                if (os.path.exists(process_en_cours[0] + '.pdf')):
                    os.remove(process_en_cours[0] + '.pdf')
                    etat_traitement.append("Traité")

                else :
                    print("Problème traitement")
                    etat_traitement.append("Pas traité")
            
            arret_timer = True
            time.sleep(1)
            arret_timer = False
            test_langue_sep(dossier_destination)
            
            print("Fin de phase !")
            pd.DataFrame({domaine_list[indice_domaine] : memoire, "Etat" : etat_traitement}).to_csv(domaine_list[indice_domaine] + " memoire.csv")

        domaine_list[indice_domaine] = "Traité"

In [191]:
#os.makedirs("/content/lot 1/")

In [192]:
#shutil.rmtree("Thèse mal lues/")
#shutil.rmtree("Thèse extraite/")

In [195]:
dossier_destination = "Thèse extraite"
dossier_mal_lu = "Thèse mal lue"
extraction(data, 3, False)

tel-02789591v1.pdf


  2%|▏         | 3/169 [00:00<00:29,  5.70it/s]

tel-02789793v1.pdf


  4%|▍         | 7/169 [00:01<00:25,  6.46it/s]

tel-00275947v1.pdf


...

 22%|██▏       | 45/206 [00:05<00:26,  6.08it/s]

tel-00012032v1.pdf


 47%|████▋     | 123/262 [01:27<01:26,  1.60it/s]Process Process-485:125:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-173-ba1c4fc4abb8>", line 3, in extraction_remplissage
    tuple_inserer = [page.extract_text(), nom, num_page]
  File "/home/eisti/.local/lib/python3.6/site-packages/pdfplumber/page.py", line 248, in extract_text
    return utils.extract_text(self.chars, **kwargs)
  File "/home/eisti/.local/lib/python3.6/site-packages/pdfplumber/container.py", line 32, in chars
    return self.objects.get("char", [])
  File "/home/eisti/.local/lib/python3.6/site-packages/pdfplumber/page.py", line 152, in objects
    self._objects = self.parse_objects()
  File "/home/eisti/.local/lib/python3.6/site-packages/pdfplumber/page.py", line 210, in parse_objects
    for ob

KeyboardInterrupt: 

  File "<ipython-input-175-7892745a6108>", line 26, in traitement_pages
    page_textes_paire.append(queue_paire.get())
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 94, in get
    res = self._recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
 47%|████▋     | 124/262 [13:23<14:54,  6.48s/it]
